<a href="https://colab.research.google.com/github/ReethikaNaramalla/icp5/blob/main/icp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data = pd.read_csv('/content/Sentiment.csv')
path_to_csv = '/content/Sentiment.csv'

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

291/291 - 67s - loss: 0.8270 - accuracy: 0.6433 - 67s/epoch - 229ms/step
144/144 - 2s - loss: 0.7525 - accuracy: 0.6647 - 2s/epoch - 12ms/step
0.752544641494751
0.6647007465362549
['loss', 'accuracy']


In [ ]:
model.save('sentiment_model.h5')

In [ ]:
from keras.models import load_model
import numpy as np

loaded_model = load_model('sentiment_model.h5')

new_text = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"]
new_text = tokenizer.texts_to_sequences(new_text)
new_text = pad_sequences(new_text, maxlen=X.shape[1], dtype='int32', value=0)
sentiment_prob = loaded_model.predict(new_text, batch_size=1, verbose=2)[0]

sentiment_classes = ['Negative', 'Neutral', 'Positive']
sentiment_pred = sentiment_classes[np.argmax(sentiment_prob)]

print("Predicted sentiment: ", sentiment_pred)
print("Predicted probabilities: ", sentiment_prob)

1/1 - 0s - 267ms/epoch - 267ms/step
Predicted sentiment:  Positive
Predicted probabilities:  [0.3960901  0.13720441 0.4667054 ]


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam

def create_model(units=196, dropout=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(units, dropout=dropout, recurrent_dropout=dropout))
    model.add(Dense(3, activation='softmax'))
    optimizer = Adam(lr=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=2)

units = [64, 128, 196]
dropout = [0.1, 0.2, 0.3]
learning_rate = [0.001, 0.01, 0.1]
epochs = [1]
batch_size = [32]

param_grid = dict(units=units, dropout=dropout, learning_rate=learning_rate, epochs=epochs, batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2)
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

194/194 - 28s - loss: 0.8531 - accuracy: 0.6318 - 28s/epoch - 142ms/step
97/97 - 2s - loss: 0.8095 - accuracy: 0.6488 - 2s/epoch - 18ms/step


[CV] END batch_size=32, dropout=0.1, epochs=1, learning_rate=0.001, units=64; total time=  45.3s
194/194 - 28s - loss: 0.8527 - accuracy: 0.6333 - 28s/epoch - 143ms/step
97/97 - 2s - loss: 0.7851 - accuracy: 0.6697 - 2s/epoch - 17ms/step


[CV] END batch_size=32, dropout=0.1, epochs=1, learning_rate=0.001, units=64; total time=  29.9s
194/194 - 28s - loss: 0.8492 - accuracy: 0.6371 - 28s/epoch - 146ms/step
97/97 - 2s - loss: 0.7673 - accuracy: 0.6720 - 2s/epoch - 16ms/step


[CV] END batch_size=32, dropout=0.1, epochs=1, learning_rate=0.001, units=128; total time=  30.0s
194/194 - 28s - loss: 0.8489 - accuracy: 0.6324 - 28s/epoch - 146ms/step
97/97 - 1s - loss: 0.7816 - accuracy: 0.6711 - 1s/epoch - 13ms/step


[CV] END batch_size=32, dropout=0.1, epochs=1, learning_rate=0.001, units=128; total time=  45.5s
194/194 - 29s - loss: 0.8463 - accuracy: 0.6319 - 29s/epoch - 150ms/step
97/97 - 1s - loss: 0.7677 - accuracy: 0.6587 - 1s/epoch - 13ms/step


...

[CV] END batch_size=32, dropout=0.3, epochs=1, learning_rate=0.1, units=128; total time=  31.2s
194/194 - 29s - loss: 1.2358 - accuracy: 0.5207 - 29s/epoch - 151ms/step
97/97 - 1s - loss: 1.3183 - accuracy: 0.5780 - 1s/epoch - 13ms/step


[CV] END batch_size=32, dropout=0.3, epochs=1, learning_rate=0.1, units=128; total time=  31.1s
194/194 - 31s - loss: 1.7316 - accuracy: 0.4975 - 31s/epoch - 157ms/step
97/97 - 1s - loss: 1.2731 - accuracy: 0.5371 - 1s/epoch - 13ms/step


[CV] END batch_size=32, dropout=0.3, epochs=1, learning_rate=0.1, units=196; total time=  46.6s
194/194 - 30s - loss: 1.8211 - accuracy: 0.5020 - 30s/epoch - 153ms/step
97/97 - 1s - loss: 1.3079 - accuracy: 0.4374 - 1s/epoch - 13ms/step


[CV] END batch_size=32, dropout=0.3, epochs=1, learning_rate=0.1, units=196; total time=  31.5s
194/194 - 29s - loss: 1.4221 - accuracy: 0.5121 - 29s/epoch - 148ms/step
97/97 - 2s - loss: 1.2854 - accuracy: 0.5347 - 2s/epoch - 16ms/step


[CV] END batch_size=32, dropout=0.3, epochs=1, learning_rate=0.1, units=196; total time=  45.4s
291/291 - 43s - loss: 0.8174 - accuracy: 0.6496 - 43s/epoch - 149ms/step
Best: 0.670827 using {'batch_size': 32, 'dropout': 0.3, 'epochs': 1, 'learning_rate': 0.01, 'units': 64}
